In [2]:
# define imports
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
#import pandas as pd
#import PIL
import os, random

from glob import glob
from mpl_toolkits.axes_grid1 import ImageGrid
from IPython.display import display, Image
from IPython.core.display import HTML



import copy
#import cv2                

from io import open
import json
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os, os.path, random
from PIL import Image
import requests
import shutil
import time

import torch
from torch.autograd import Variable
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms


%matplotlib inline


# my imports

from os import listdir
from os.path import isfile, join, abspath
from style_transfer.stylize import transfer


In [4]:
# All images are resized to 224x224 and normalized
# Only training images receive further augmentation

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),    
}


In [5]:
data_dir = 'dogImages_style_transfer'
print(os.path.join(data_dir, 'valid'))


# we create some dictionaries
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'valid', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=5,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'valid', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid', 'test']}
print(dataset_sizes)
class_names = image_datasets['train'].classes
n_classes = len(class_names)

dogImages_style_transfer/valid
{'train': 59800, 'valid': 835, 'test': 836}


In [6]:
#import pandas as pd
import boto3
import sagemaker

In [7]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

In [8]:
import os

# this is the S3 data_dir name
data_dir = 'dogImages'
train_dir = 'train'

# set prefix, a descriptive name for a directory  
prefix = 'capstone-project'

# upload data for the first time; otherwise use the second expression (because the S3 upload takes some time)
#train_location = sagemaker_session.upload_data(os.path.join(data_dir), key_prefix=prefix)
train_location = 's3://{}/{}/'.format(bucket, prefix) 

In [9]:
# your import and estimator code, here
from sagemaker.pytorch import PyTorch

output_path = 's3://{}/{}'.format(bucket, prefix)

# note: this PyTorch estimator is specified with code that includes
# the preprocessing with the imgaug library and is trained on the style
# transferred images
source_dir = 'source_pytorch'

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir=source_dir,
                    role=role,
                    framework_version='1.1.0',
                    train_instance_count=1,
                    #train_instance_type='ml.c4.xlarge',
                    train_instance_type='ml.p2.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'epochs': 10
                    })


In [ ]:
%%time

# Train your estimator on S3 training data
estimator.fit({'train': train_location})

2019-08-05 18:22:21 Starting - Starting the training job...
2019-08-05 18:22:23 Starting - Launching requested ML instances......
2019-08-05 18:23:23 Starting - Preparing the instances for training......
2019-08-05 18:24:39 Downloading - Downloading input data................................................
2019-08-05 18:32:50 Training - Training image download completed. Training in progress..
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-08-05 18:32:51,640 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-08-05 18:32:51,667 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-08-05 18:32:51,974 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-08-05 18:32:52,241 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2019-08-05 18:32:52,242 sagemaker-containers INFO 

Using device cuda.
Get train data loader.
Epoch: 1 #011Training Loss: 4.043044 #011Validation Loss: 1.632729
Epoch: 2 #011Training Loss: 3.273106 #011Validation Loss: 1.045015
Epoch: 3 #011Training Loss: 3.010815 #011Validation Loss: 0.834669
Epoch: 4 #011Training Loss: 2.873254 #011Validation Loss: 0.721248
Epoch: 5 #011Training Loss: 2.783619 #011Validation Loss: 0.661954
Epoch: 6 #011Training Loss: 2.719921 #011Validation Loss: 0.619580
Epoch: 7 #011Training Loss: 2.670455 #011Validation Loss: 0.589056
Epoch: 8 #011Training Loss: 2.636757 #011Validation Loss: 0.563158
Epoch: 9 #011Training Loss: 2.604821 #011Validation Loss: 0.545320


In [16]:
%%time

# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

print(estimator.model_data)

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='1.1.0',
                     entry_point='predict.py',
                     source_dir=source_dir)


s3://sagemaker-eu-central-1-505649883860/capstone-project/sagemaker-pytorch-2019-08-05-18-22-20-825/output/model.tar.gz
CPU times: user 19.4 ms, sys: 98 µs, total: 19.4 ms
Wall time: 91.6 ms


In [17]:
# ATTENTION: Please use the ml.t2.large instance here. Otherwise the deployment will fail because of resource constraints
# predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.large')



-------------------------------------------------------------------------------------------------!

In [18]:
def test_acc(test_set, predictor):
    ''' 
    This method calculates our accuracy metric. For this, it will send 
    batches of data to the endpoint and receive the predictions
    '''
    test_acc = 0.0
    running_corrects = 0

    for inputs, labels in test_set:
        print(labels)
        test_y_preds = predictor.predict(inputs)

        _, preds_tensor = torch.max(torch.from_numpy(test_y_preds), 1)
        preds = np.squeeze(preds_tensor.numpy()) if not use_cuda else np.squeeze(preds_tensor.cpu().numpy())

        print(f"preds is {preds}")
    
        # compare the batch of predictions with the ground truth of labels
        running_corrects += torch.sum(torch.from_numpy(preds) == labels)

    test_acc = running_corrects.double() / len(test_set.dataset)

    print('Acc: {:.2f}%'.format(test_acc*100))

    return test_acc

use_cuda = torch.cuda.is_available()
testacc=test_acc(iter(dataloaders['test']), predictor)
print(testacc)

tensor([ 15,   6,  18,  61, 124])
preds is [15  6 18 61 25]
tensor([ 47,   1, 132,  70, 108])
preds is [ 47  61 132  70 108]
tensor([ 96,  40, 110,  36,  27])
preds is [124  40 110  36  27]
tensor([ 21,  29, 111,  38,   0])
preds is [ 21  29 111  38   0]
tensor([ 55,  40,  95,  16, 102])
preds is [55 40 49 16 40]
tensor([ 33,  90, 123,   3,  92])
preds is [ 33  90 123  44  73]
tensor([ 75,  66,  23,  40, 114])
preds is [ 75  66  23  43 114]
tensor([ 18,  35, 120,  80, 121])
preds is [ 18  35 120  80  36]
tensor([ 97, 106, 124, 114,  87])
preds is [ 97 106 124 114  87]
tensor([111,  13,  86,  72,   7])
preds is [111  13  86  73   7]
tensor([ 20,  58, 105, 104,  45])
preds is [ 20  17 105  71  45]
tensor([ 75, 114,  30, 109,  53])
preds is [ 75 114  30 117  53]
tensor([132,  17, 129,  61,  14])
preds is [132  17 129  61  14]
tensor([ 62,   9, 132,   5,  30])
preds is [ 62   9 132   5  53]
tensor([ 88,   6,  77, 123,  55])
preds is [ 88  15  77 123 118]
tensor([ 10, 126,  15,  36,  35])
p

preds is [106 100  31  91  29]
tensor([16, 56, 20, 42, 61])
preds is [ 16  56  20  75 129]
tensor([42, 95, 15, 88, 23])
preds is [42 95 15 74 23]
tensor([ 97,  39,  20, 115,  10])
preds is [ 97  39  20 115 117]
tensor([108,  37, 100,  70,  85])
preds is [ 91 132 100  70  85]
tensor([ 30,  14, 122,  19, 104])
preds is [ 30  14 114  19 104]
tensor([107,   3,  18,  17,  43])
preds is [107   3  18  58   7]
tensor([ 41, 122,  40,  95,  26])
preds is [ 41 122  40  95  26]
tensor([77,  0, 31,  0, 88])
preds is [77 98 31  0 88]
tensor([50, 57,  2,  9,  4])
preds is [50 57  2  9  4]
tensor([78, 89, 41, 38, 46])
preds is [ 78  89 110  38  46]
tensor([ 39, 118,  59, 118, 102])
preds is [ 39 118  59  16 102]
tensor([19, 74, 98, 42, 80])
preds is [ 19 113  98  42  89]
tensor([ 20,  37,  11,  89, 111])
preds is [ 20  37  11 119 111]
tensor([ 38,  65, 128,  44,  47])
preds is [38 65 50 44 47]
tensor([ 4, 63, 81, 13, 27])
preds is [ 4 63 81 13 27]
tensor([ 64,  13, 125, 129, 114])
preds is [ 64  13 12